# `PromptBuilder`
- Enable GPU in Docker [https://github.com/ollama/ollama/blob/main/docs/docker.md](https://github.com/ollama/ollama/blob/main/docs/docker.md):
  ```bash
  curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey \
      | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg
  curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list \
      | sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' \
      | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list
  sudo apt-get update

  sudo apt-get install -y nvidia-container-toolkit

  sudo nvidia-ctk runtime configure --runtime=docker

  sudo systemctl restart docker
  ```

- Run Ollama in Docker integrated with GPU:
  ```bash
  docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
  docker exec ollama ollama pull hf.co/janhq/Vistral-7b-Chat-GGUF:Q5_K_M
  docker exec ollama ollama pull hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0
  ```

In [1]:
%load_ext autoreload
%autoreload 2

## On its own

In [2]:
from haystack.components.builders import PromptBuilder

In [3]:
template = "Translate the following context to {{ target_language }}. Context: {{ snippet }}; Translation:"
builder = PromptBuilder(template=template)
res = builder.run(target_language="spanish", snippet="I can't speak spanish.")

In [4]:
res

{'prompt': "Translate the following context to spanish. Context: I can't speak spanish.; Translation:"}

## In pipeline

In [5]:
from haystack import Pipeline, Document
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.components.builders.prompt_builder import PromptBuilder

In [6]:
# in a real world use case documents could come from a retriever, web, or any other source
documents = [Document(content="Cường sống ở thành phố Hồ Chí Minh"), Document(content="Cường là một kĩ sư phần mềm")]
prompt_template = """
    Đưa ra những tài liệu này, trả lời câu hỏi.\nTài liệu:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nCâu hỏi: {{query}}
    \nTrả lời:
    """
llm = OllamaGenerator(
    model="hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0",
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True),
    url = "http://localhost:11434",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.9})

In [7]:
p = Pipeline()
p.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
p.add_component(instance=llm, name="llm")
p.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [8]:
question = "Cường sống ở đâu?"
result = p.run({"prompt_builder": {"documents": documents, "query": question}})
print(result)

Trả lời: Cường sống ở thành phố Hồ Chí Minh. {'llm': {'replies': ['Trả lời: Cường sống ở thành phố Hồ Chí Minh. '], 'meta': [{'model': 'hf.co/janhq/Vistral-7b-Chat-GGUF:Q8_0', 'created_at': '2024-12-29T14:24:44.010172429Z', 'done': False, 'done_reason': None, 'total_duration': None, 'load_duration': None, 'prompt_eval_count': None, 'prompt_eval_duration': None, 'eval_count': None, 'eval_duration': None, 'context': None}]}}


### Changing the template at runtime (Prompt Engineering)

In [9]:
documents = [
    Document(content="Cường sống ở thành phố Hồ Chí Minh", meta={"name": "doc1"}), 
    Document(content="Cường là một kĩ sư phần mềm", meta={"name": "doc1"}),
]

new_template = """
    Bạn là một trợ lý hữu ích.
    Đưa ra những tài liệu này, trả lời câu hỏi.
    Tài liệu:
    {% for doc in documents %}
        Tài liệu {{ loop.index }}:
        Tên tài liệu: {{ doc.meta['name'] }}
        {{ doc.content }}
    {% endfor %}

    Câu hỏi: {{ query }}
    Trả lời:
    """

In [10]:
res = p.run({
    "prompt_builder": {
        "documents": documents, 
        "query": question, 
        "template": new_template,
        },
    })

Cường sống tại thành phố Hồ Chí Minh.
Trả lời:
Thành phố Hồ Chí Minh.


### Overwritting variables at runtime

In [11]:
language_template = """
    Bạn là một trợ lý hữu ích.
    Đưa ra những tài liệu này, trả lời câu hỏi.
    Tài liệu:
    {% for doc in documents %}
        Tài liệu {{ loop.index }}:
        Tên tài liệu: {{ doc.meta['name'] }}
        {{ doc.content }}
    {% endfor %}

    Câu hỏi: {{ query }}
    Vui lòng trả lời câu hỏi của bằng {{ answer_language | default('Tiếng Việt') }}
    Trả lời:
    """

In [12]:
res = p.run({
    "prompt_builder": {
        "documents": documents, 
        "query": question, 
        "template": language_template, 
        "template_variables": {"answer_language": "Tiếng Việt"},
    }})

    Văn bản: Tên tài liệu 1 ghi là Cường sống ở thành phố Hồ Chí Minh và Tài liệu 2 nói rằng Cường là một kĩ sư phần mềm. Câu hỏi đặt ra về việc Cường sống ở đâu?
    Trả lời: Cường sống ở Thành phố Hồ Chí Minh.
